# Object Relational Tutorial

## Version Check

In [1]:
import sqlalchemy
sqlalchemy.__version__

'1.3.7'

## Connecting

In [2]:
from sqlalchemy import create_engine

In [3]:
engine = create_engine('sqlite:///:memory:', echo=True)

## Declare a Mapping

In [4]:
from sqlalchemy.ext.declarative import declarative_base

In [5]:
Base = declarative_base()

In [6]:
from sqlalchemy import Column, Integer, String

In [7]:
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    
    def __repr__(self):
        return "<User(name='{}', fullname='{}', nickname='{}')>".format(
        self.name, self.fullname, self.nickname)

## Create a Schema

In [8]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

In [9]:
Base.metadata.create_all(engine)

2019-09-28 09:17:33,308 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-09-28 09:17:33,311 INFO sqlalchemy.engine.base.Engine ()
2019-09-28 09:17:33,313 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-09-28 09:17:33,313 INFO sqlalchemy.engine.base.Engine ()
2019-09-28 09:17:33,314 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2019-09-28 09:17:33,315 INFO sqlalchemy.engine.base.Engine ()
2019-09-28 09:17:33,315 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2019-09-28 09:17:33,316 INFO sqlalchemy.engine.base.Engine ()
2019-09-28 09:17:33,317 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2019-09-28 09:17:33,317 INFO sqlalchemy.engine.base.Engine ()
2019-09-28 09:17:33,318 INFO sqlalchemy.engine.base.Engine COMMIT


## Create an Instance of the Mapped Class

In [10]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='ednickname')

## Creating a Session

In [11]:
from sqlalchemy.orm import sessionmaker

In [12]:
Session = sessionmaker(bind=engine)

In [13]:
session = Session()

## Adding and Updating Objects

In [14]:
session.add(ed_user)

In [15]:
our_user = session.query(User).filter_by(name='ed').first()

2019-09-28 09:17:33,375 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-09-28 09:17:33,377 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-09-28 09:17:33,378 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'ednickname')
2019-09-28 09:17:33,379 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2019-09-28 09:17:33,380 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


In [16]:
ed_user is our_user

True

In [17]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', nickname='windy'),
    User(name='mary', fullname='Mary Contrary', nickname='mary'),
    User(name='fred', fullname='Fred Flintstone', nickname='freddy')
])

In [18]:
ed_user.nickname = 'eddie'

In [19]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', nickname='eddie')>])

In [20]:
session.new

IdentitySet([<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <User(name='mary', fullname='Mary Contrary', nickname='mary')>, <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>])

In [21]:
session.commit()

2019-09-28 09:17:33,431 INFO sqlalchemy.engine.base.Engine UPDATE users SET nickname=? WHERE users.id = ?
2019-09-28 09:17:33,432 INFO sqlalchemy.engine.base.Engine ('eddie', 1)
2019-09-28 09:17:33,433 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-09-28 09:17:33,433 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'windy')
2019-09-28 09:17:33,434 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-09-28 09:17:33,435 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'mary')
2019-09-28 09:17:33,435 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-09-28 09:17:33,436 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flintstone', 'freddy')
2019-09-28 09:17:33,437 INFO sqlalchemy.engine.base.Engine COMMIT


In [22]:
ed_user.id

2019-09-28 09:17:33,444 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-09-28 09:17:33,446 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2019-09-28 09:17:33,446 INFO sqlalchemy.engine.base.Engine (1,)


1

## Rolling Back

In [23]:
ed_user.name = 'Edwardo'

In [24]:
fake_user = User(name='fakeuser', fullname='Invalid', nickname='12345')
session.add(fake_user)

In [25]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2019-09-28 09:17:33,474 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.id = ?
2019-09-28 09:17:33,474 INFO sqlalchemy.engine.base.Engine ('Edwardo', 1)
2019-09-28 09:17:33,476 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-09-28 09:17:33,476 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '12345')
2019-09-28 09:17:33,477 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2019-09-28 09:17:33,478 INFO sqlalchemy.engine.base.Engine ('Edwardo', 'fakeuser')


[<User(name='Edwardo', fullname='Ed Jones', nickname='eddie')>,
 <User(name='fakeuser', fullname='Invalid', nickname='12345')>]

In [26]:
session.rollback()

2019-09-28 09:17:33,486 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [27]:
ed_user.name

2019-09-28 09:17:33,493 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-09-28 09:17:33,496 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2019-09-28 09:17:33,497 INFO sqlalchemy.engine.base.Engine (1,)


'ed'

In [28]:
fake_user in session

False

In [29]:
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

2019-09-28 09:17:33,515 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2019-09-28 09:17:33,516 INFO sqlalchemy.engine.base.Engine ('ed', 'fakeuser')


[<User(name='ed', fullname='Ed Jones', nickname='eddie')>]

## Querying

In [30]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

2019-09-28 09:17:33,526 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
2019-09-28 09:17:33,527 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [31]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2019-09-28 09:17:33,534 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2019-09-28 09:17:33,534 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [32]:
for row in session.query(User, User.name).all():
    print(row.User, row.name)

2019-09-28 09:17:33,542 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users
2019-09-28 09:17:33,544 INFO sqlalchemy.engine.base.Engine ()
<User(name='ed', fullname='Ed Jones', nickname='eddie')> ed
<User(name='wendy', fullname='Wendy Williams', nickname='windy')> wendy
<User(name='mary', fullname='Mary Contrary', nickname='mary')> mary
<User(name='fred', fullname='Fred Flintstone', nickname='freddy')> fred


In [33]:
for row in session.query(User.name.label('name_label')).all():
    print(row.name_label)

2019-09-28 09:17:33,554 INFO sqlalchemy.engine.base.Engine SELECT users.name AS name_label 
FROM users
2019-09-28 09:17:33,555 INFO sqlalchemy.engine.base.Engine ()
ed
wendy
mary
fred


In [34]:
from sqlalchemy.orm import aliased

In [35]:
user_alias = aliased(User, name='user_alias')

In [36]:
for row in session.query(user_alias, user_alias.name).all():
    print(row.user_alias)

2019-09-28 09:17:33,574 INFO sqlalchemy.engine.base.Engine SELECT user_alias.id AS user_alias_id, user_alias.name AS user_alias_name, user_alias.fullname AS user_alias_fullname, user_alias.nickname AS user_alias_nickname 
FROM users AS user_alias
2019-09-28 09:17:33,575 INFO sqlalchemy.engine.base.Engine ()
<User(name='ed', fullname='Ed Jones', nickname='eddie')>
<User(name='wendy', fullname='Wendy Williams', nickname='windy')>
<User(name='mary', fullname='Mary Contrary', nickname='mary')>
<User(name='fred', fullname='Fred Flintstone', nickname='freddy')>


In [37]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

2019-09-28 09:17:33,585 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2019-09-28 09:17:33,586 INFO sqlalchemy.engine.base.Engine (2, 1)
<User(name='wendy', fullname='Wendy Williams', nickname='windy')>
<User(name='mary', fullname='Mary Contrary', nickname='mary')>


In [38]:
for name, in session.query(User.name).filter_by(fullname='Ed Jones'):
    print(name)

2019-09-28 09:17:33,593 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2019-09-28 09:17:33,594 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)
ed


In [39]:
for name, in session.query(User.name).filter(User.fullname=='Ed Jones'):
    print(name)

2019-09-28 09:17:33,603 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2019-09-28 09:17:33,604 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)
ed


In [40]:
for user in session.query(User).filter(User.name=='ed').filter(User.fullname=='Ed Jones'):
    print(user)

2019-09-28 09:17:33,612 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2019-09-28 09:17:33,614 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones')
<User(name='ed', fullname='Ed Jones', nickname='eddie')>


## Returning Lists and Scalars

In [41]:
query = session.query(User).filter(User.name.like('%ed')).order_by(User.id)
query.all()

2019-09-28 09:17:33,623 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2019-09-28 09:17:33,624 INFO sqlalchemy.engine.base.Engine ('%ed',)


[<User(name='ed', fullname='Ed Jones', nickname='eddie')>,
 <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>]

In [42]:
query.first()

2019-09-28 09:17:33,633 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
 LIMIT ? OFFSET ?
2019-09-28 09:17:33,634 INFO sqlalchemy.engine.base.Engine ('%ed', 1, 0)


<User(name='ed', fullname='Ed Jones', nickname='eddie')>

In [43]:
user = query.one()

2019-09-28 09:17:33,643 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2019-09-28 09:17:33,644 INFO sqlalchemy.engine.base.Engine ('%ed',)


MultipleResultsFound: Multiple rows were found for one()

In [44]:
query = session.query(User.id).filter(User.name == 'ed').order_by(User.id)

In [45]:
query.scalar()

2019-09-28 09:17:55,676 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id 
FROM users 
WHERE users.name = ? ORDER BY users.id
2019-09-28 09:17:55,677 INFO sqlalchemy.engine.base.Engine ('ed',)


1

## Using Textual SQL

In [46]:
from sqlalchemy import text

In [47]:
for user in session.query(User).filter(text("id < 10")).order_by(text("id")).all():
    print(user.name)

2019-09-28 09:17:57,062 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE id < 10 ORDER BY id
2019-09-28 09:17:57,063 INFO sqlalchemy.engine.base.Engine ()
ed
wendy
mary
fred


In [48]:
session.query(User).filter(text("id < :value and name = :name")).params(value = 10, name = 'fred').order_by(User.id).one()

2019-09-28 09:17:57,314 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE id < ? and name = ? ORDER BY users.id
2019-09-28 09:17:57,315 INFO sqlalchemy.engine.base.Engine (10, 'fred')


<User(name='fred', fullname='Fred Flintstone', nickname='freddy')>

In [49]:
session.query(User).from_statement(text(
    "SELECT * FROM users WHERE name = :name"
)).params(name = 'ed').all()

2019-09-28 09:17:57,551 INFO sqlalchemy.engine.base.Engine SELECT * FROM users WHERE name = ?
2019-09-28 09:17:57,552 INFO sqlalchemy.engine.base.Engine ('ed',)


[<User(name='ed', fullname='Ed Jones', nickname='eddie')>]

In [50]:
stmt = text("SELECT name, id, fullname, nickname FROM users WHERE name = :name")
stmt = stmt.columns(User.name, User.id, User.fullname, User.nickname)

session.query(User).from_statement(stmt).params(name='ed').all()

2019-09-28 09:17:57,988 INFO sqlalchemy.engine.base.Engine SELECT name, id, fullname, nickname FROM users WHERE name = ?
2019-09-28 09:17:57,989 INFO sqlalchemy.engine.base.Engine ('ed',)


[<User(name='ed', fullname='Ed Jones', nickname='eddie')>]

In [51]:
stmt = text("SELECT name, id FROM users WHERE name = :name")
stmt = stmt.columns(User.name, User.id)

session.query(User.id, User.name).from_statement(stmt).params(name='ed').all()

2019-09-28 09:17:58,289 INFO sqlalchemy.engine.base.Engine SELECT name, id FROM users WHERE name = ?
2019-09-28 09:17:58,289 INFO sqlalchemy.engine.base.Engine ('ed',)


[(1, 'ed')]

## Counting

In [52]:
session.query(User).filter(User.name.like('%ed')).count()

2019-09-28 09:17:59,055 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ?) AS anon_1
2019-09-28 09:17:59,056 INFO sqlalchemy.engine.base.Engine ('%ed',)


2

In [53]:
from sqlalchemy import func

In [54]:
session.query(func.count(User.name), User.name).group_by(User.name).all()

2019-09-28 09:17:59,472 INFO sqlalchemy.engine.base.Engine SELECT count(users.name) AS count_1, users.name AS users_name 
FROM users GROUP BY users.name
2019-09-28 09:17:59,473 INFO sqlalchemy.engine.base.Engine ()


[(1, 'ed'), (1, 'fred'), (1, 'mary'), (1, 'wendy')]

In [55]:
session.query(func.count('*')).select_from(User).scalar()

2019-09-28 09:17:59,897 INFO sqlalchemy.engine.base.Engine SELECT count(?) AS count_1 
FROM users
2019-09-28 09:17:59,898 INFO sqlalchemy.engine.base.Engine ('*',)


4

In [56]:
session.query(func.count(User.id)).scalar()

2019-09-28 09:18:00,154 INFO sqlalchemy.engine.base.Engine SELECT count(users.id) AS count_1 
FROM users
2019-09-28 09:18:00,155 INFO sqlalchemy.engine.base.Engine ()


4

## Building a Relationship

In [57]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

In [58]:
class Address(Base):
    __tablename__ = 'addresses'
    
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return "<Address(email_address={})>".format(self.email_address)

In [59]:
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")

In [60]:
Base.metadata.create_all(engine)

2019-09-28 09:18:01,845 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2019-09-28 09:18:01,846 INFO sqlalchemy.engine.base.Engine ()
2019-09-28 09:18:01,848 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("addresses")
2019-09-28 09:18:01,848 INFO sqlalchemy.engine.base.Engine ()
2019-09-28 09:18:01,849 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("addresses")
2019-09-28 09:18:01,850 INFO sqlalchemy.engine.base.Engine ()
2019-09-28 09:18:01,852 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2019-09-28 09:18:01,853 INFO sqlalchemy.engine.base.Engine ()
2019-09-28 09:18:01,854 INFO sqlalchemy.engine.base.Engine COMMIT


In [61]:
jack = User(name='jack', fullname='Jack Bean', nickname='jacknick')
jack.addresses

[]

In [62]:
jack.addresses = [Address(email_address='jack@gmail.com'),
                 Address(email_address='jack@yahoo.com')]

In [63]:
jack.addresses[1]

<Address(email_address=jack@yahoo.com)>

In [64]:
jack.addresses[1].user

<User(name='jack', fullname='Jack Bean', nickname='jacknick')>

In [65]:
session.add(jack)
session.commit()

2019-09-28 09:18:03,669 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-09-28 09:18:03,670 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Bean', 'jacknick')
2019-09-28 09:18:03,672 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2019-09-28 09:18:03,673 INFO sqlalchemy.engine.base.Engine ('jack@gmail.com', 5)
2019-09-28 09:18:03,674 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2019-09-28 09:18:03,674 INFO sqlalchemy.engine.base.Engine ('jack@yahoo.com', 5)
2019-09-28 09:18:03,676 INFO sqlalchemy.engine.base.Engine COMMIT


In [66]:
jack = session.query(User).filter_by(name='jack').one()
jack

2019-09-28 09:18:04,222 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-09-28 09:18:04,224 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-09-28 09:18:04,225 INFO sqlalchemy.engine.base.Engine ('jack',)


<User(name='jack', fullname='Jack Bean', nickname='jacknick')>

In [67]:
jack.addresses

2019-09-28 09:18:04,870 INFO sqlalchemy.engine.base.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE ? = addresses.user_id ORDER BY addresses.id
2019-09-28 09:18:04,871 INFO sqlalchemy.engine.base.Engine (5,)


[<Address(email_address=jack@gmail.com)>,
 <Address(email_address=jack@yahoo.com)>]

## Querying with Joins

In [71]:
for u, a in session.query(User, Address).\
                            filter(User.id == Address.user_id).filter(Address.email_address == 'jack@gmail.com').all():
    print(u)
    print(a)
    

2019-09-28 09:25:09,725 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM users, addresses 
WHERE users.id = addresses.user_id AND addresses.email_address = ?
2019-09-28 09:25:09,726 INFO sqlalchemy.engine.base.Engine ('jack@gmail.com',)
<User(name='jack', fullname='Jack Bean', nickname='jacknick')>
<Address(email_address=jack@gmail.com)>


In [72]:
session.query(User).join(Address).filter(Address.email_address == 'jack@gmail.com').all()

2019-09-28 09:27:44,072 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users JOIN addresses ON users.id = addresses.user_id 
WHERE addresses.email_address = ?
2019-09-28 09:27:44,073 INFO sqlalchemy.engine.base.Engine ('jack@gmail.com',)


[<User(name='jack', fullname='Jack Bean', nickname='jacknick')>]

## Using Aliases

In [73]:
from sqlalchemy.orm import aliased

In [74]:
adalias1 = aliased(Address)
adalias2 = aliased(Address)

In [76]:
for username, email1, email2 in session.query(User.name, adalias1.email_address, adalias2.email_address).\
                                        join(adalias1, User.addresses).join(adalias2, User.addresses).\
                                        filter(adalias1.email_address == 'jack@gmail.com').\
                                        filter(adalias2.email_address == 'jack@yahoo.com'):
    print(username, email1, email2)

2019-09-28 09:35:38,124 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, addresses_1.email_address AS addresses_1_email_address, addresses_2.email_address AS addresses_2_email_address 
FROM users JOIN addresses AS addresses_1 ON users.id = addresses_1.user_id JOIN addresses AS addresses_2 ON users.id = addresses_2.user_id 
WHERE addresses_1.email_address = ? AND addresses_2.email_address = ?
2019-09-28 09:35:38,125 INFO sqlalchemy.engine.base.Engine ('jack@gmail.com', 'jack@yahoo.com')
jack jack@gmail.com jack@yahoo.com


## Using Subqueries

In [77]:
from sqlalchemy.sql import func

In [78]:
stmt = session.query(Address.user_id, func.count('*').label('address_count')).group_by(Address.user_id).subquery()

In [79]:
for u, count in session.query(User, stmt.c.address_count).\
                        outerjoin(stmt, User.id == stmt.c.address_count).order_by(User.id):
    print(u, count)

2019-09-28 09:41:26,210 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, anon_1.address_count AS anon_1_address_count 
FROM users LEFT OUTER JOIN (SELECT addresses.user_id AS user_id, count(?) AS address_count 
FROM addresses GROUP BY addresses.user_id) AS anon_1 ON users.id = anon_1.address_count ORDER BY users.id
2019-09-28 09:41:26,211 INFO sqlalchemy.engine.base.Engine ('*',)
<User(name='ed', fullname='Ed Jones', nickname='eddie')> None
<User(name='wendy', fullname='Wendy Williams', nickname='windy')> 2
<User(name='mary', fullname='Mary Contrary', nickname='mary')> None
<User(name='fred', fullname='Fred Flintstone', nickname='freddy')> None
<User(name='jack', fullname='Jack Bean', nickname='jacknick')> None


## Selecting Entities from Subqueries

In [80]:
stmt = session.query(Address).filter(Address.email_address != 'jack@yahoo.com').subquery()

In [81]:
adalias = aliased(Address, stmt)

In [83]:
for user, address in session.query(User, adalias).join(adalias, User.addresses):
    print(user)
    print(address)

2019-09-28 09:45:15,628 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, anon_1.id AS anon_1_id, anon_1.email_address AS anon_1_email_address, anon_1.user_id AS anon_1_user_id 
FROM users JOIN (SELECT addresses.id AS id, addresses.email_address AS email_address, addresses.user_id AS user_id 
FROM addresses 
WHERE addresses.email_address != ?) AS anon_1 ON users.id = anon_1.user_id
2019-09-28 09:45:15,629 INFO sqlalchemy.engine.base.Engine ('jack@yahoo.com',)
<User(name='jack', fullname='Jack Bean', nickname='jacknick')>
<Address(email_address=jack@gmail.com)>


## Using EXISTS

In [84]:
from sqlalchemy.sql import exists

In [85]:
stmt = exists().where(Address.user_id == User.id)

In [ ]:
for name, in session.query()